Graph Reset code

In [24]:
tf.reset_default_graph()

# Lab12-4_RNN with long sequences Stacked RNN + Softmax layer

RNN with sequences의 완성이라고 할 수 있다!<br>
__우리가 배운 RNN 에 여러 층의 RNN과 Softmax layer를 추가해 보겠다.!__

In [7]:
import tensorflow as tf
import numpy as np
from tensorflow.contrib import rnn

In [25]:
tf.reset_default_graph()
sentence = ("if you want to build a ship, don't drum up people together to collect wood and don't assign them tasks and work, but rather teach them to long for the endless immensity of the sea.")

### Parameter 생성
Lab12-3 에서 배운것 그대로

In [26]:
char_set = list(set(sentence))
char_dic = {w: i for i, w in enumerate(char_set)}

data_dim = len(char_set)
hidden_size = len(char_set)
num_classes = len(char_set)
sequence_length = 10  # Any arbitrary number
learning_rate = 0.1

dataX = []
dataY = []

for i in range(0, len(sentence) - sequence_length):
    x_str = sentence[i:i + sequence_length]
    y_str = sentence[i + 1: i + sequence_length + 1]
    print(i, x_str, '->', y_str)

    x = [char_dic[c] for c in x_str]  # x str to index
    y = [char_dic[c] for c in y_str]  # y str to index

    dataX.append(x)
    dataY.append(y)

batch_size = len(dataX)


0 if you wan -> f you want
1 f you want ->  you want 
2  you want  -> you want t
3 you want t -> ou want to
4 ou want to -> u want to 
5 u want to  ->  want to b
6  want to b -> want to bu
7 want to bu -> ant to bui
8 ant to bui -> nt to buil
9 nt to buil -> t to build
10 t to build ->  to build 
11  to build  -> to build a
12 to build a -> o build a 
13 o build a  ->  build a s
14  build a s -> build a sh
15 build a sh -> uild a shi
16 uild a shi -> ild a ship
17 ild a ship -> ld a ship,
18 ld a ship, -> d a ship, 
19 d a ship,  ->  a ship, d
20  a ship, d -> a ship, do
21 a ship, do ->  ship, don
22  ship, don -> ship, don'
23 ship, don' -> hip, don't
24 hip, don't -> ip, don't 
25 ip, don't  -> p, don't d
26 p, don't d -> , don't dr
27 , don't dr ->  don't dru
28  don't dru -> don't drum
29 don't drum -> on't drum 
30 on't drum  -> n't drum u
31 n't drum u -> 't drum up
32 't drum up -> t drum up 
33 t drum up  ->  drum up p
34  drum up p -> drum up pe
35 drum up pe -> rum up peo
36

하지만, 실행을 시켜보니 잘 되지 않았다.<br>
그 이유 중 하나는, 우리가 복잡하고 많은 데이터를 다루기에는 RNN Cell 이 너무 작다 -> Wide & Deep하게 가자<br>
이렇게, wide & Deep 한 모델을 쌓아서 할 수는 없을까 해서 만든것이<br>
## 1. Stacked RNN

여러개의 RNN을 층을 두고 쌓아서 하는 것<br>
-> 코드가 크게 다를 것은 없다. 이전과 똑같은데

In [27]:
X = tf.placeholder(tf.int32, [None, sequence_length])
Y = tf.placeholder(tf.int32, [None, sequence_length])

# One-hot encoding
X_one_hot = tf.one_hot(X, num_classes)
print(X_one_hot)  # check out the shape


# Make a lstm cell with hidden_size (each unit output vector size)
# rnn.BasicLSTMCell 이거는 한개 층의 cell 을 만드는것.
def lstm_cell():
    cell = rnn.BasicLSTMCell(hidden_size, state_is_tuple=True)
    # 여기서 가장 중요한 것은  hidden_size!!
    return cell

# Tensorflow 에는  MultiRNNCell 이라는 아름다운 함수가 있다 !
# -> 우리가 기본 Cell을 만들어 놓고 
# cell = rnn.MuliRNNCell([cell] * 2, state_is_tuple = True) 만 해 주면 두개 층의 RNN이 되어버린다.
# 단순히 곱셈 한번으로 층이 쌓아지는 것!
multi_cells = rnn.MultiRNNCell([lstm_cell() for _ in range(2)], state_is_tuple=True)
# 나머지는 사실상 똑같다..

# outputs: unfolding size x hidden size, state = hidden size
outputs, _states = tf.nn.dynamic_rnn(multi_cells, X_one_hot, dtype=tf.float32)

Tensor("one_hot:0", shape=(?, 10, 25), dtype=float32)


## 2. Softmax
우리가, 앞에서 fullyconnected를 쓰는 것을 배웠다.-> 성능을 더 좋게 하기 위해서!
이와 같이, RNN으로 나온 값을 그냥 쓰지 말고, Fully connected 할때처럼 Softmax를 붙여주자! 라는 아이디어.
<br>
<br>
Softmax를 붙이는 방법? <br><br>
1) 각각의 RNN에 맞게 Softmax를 만들어 붙여줘도 좋고 (but, 메모리 소모...+Weight )<br>
2) 그림이 옆으로 이어져서 그렇지, 사실상 하나의 Layer이다 -> 각각의 Softmax를 만들어 줄 것이 아니라, 그냥 결과로 다 나온 데이터를 쭉 쌓아서 만들어 준 후 Softmax를 적용한 후 다시 풀어주면 될 것.
<br>
<br>
그래서,Softmax를 디자인 하는 입장에서는 각각의 RNN에서 나온 Data를 합친걸 들어온 입력을 처리 후 다시 reshape해줘야 하는데, 그때의 Weight을 디자인 한다고 생각하면 됨
<br>
<br>
~~~
X_for_softmax = tf.reshape(outputs, [-1,hidden_size])
outputs으로 도출된 data를 hidden_size에 맞게 reshape 해준다.
(여기서의 outputs는 RNN의 outputs)
~~~
~~~
outputs = tf.reshape(outputs, [batch_size, seq_length, num_classes])
softmax의 결과를 각각의 class 사이즈에 맞춰(output의 개수) seq_length 사이즈로
(여기서의  outputs는 위의 softmax의 outputs)
~~~

In [28]:
# (optional) softmax Layer
X_for_softmax = tf.reshape(outputs, [-1, hidden_size])
# outputs -> RNN의  outputs

## -- 아래 코드는 그 다음의 한 줄로 변경 가능 --
softmax_w = tf.get_variable("softmax_w",[hidden_size,num_classes])
# size : [inputs,outputs]
softmax_b = tf.get_variable("softmax_b",[num_classes])
# size : [num_classes]
outputs = tf.matmul(X_for_softmax, softmax_w) + softmax_b
# outputs = tf.contrib.layers.fully_connected(X_for_softmax, num_classes, activation_fn=None)

outputs = tf.reshape(outputs, [batch_size, sequence_length, num_classes])
# RNN의 output과 같게 해준다.

### Loss

In [31]:
# All weights are 1 (equal weights)
weights = tf.ones([batch_size, sequence_length])

sequence_loss = tf.contrib.seq2seq.sequence_loss(
    logits=outputs, targets=Y, weights=weights)
# 여기서 주의할것은, logits 에다가 들어가는 outputs
# 우리는 위의 코드에서, outputs을 계산할때 activation  function을 거치지 않고, matmul 만 시켜주었다.
# 이렇게, Activation function을 거치지 않고 바로 넣는 것이 맞다 !
# 만약 hidden layer에서 activation function이 적용된 값을 넣으면 문제가 있는것
# activation을 적용하지 않은 logit을 넣어야 좋은 성능을 얻을 수 있다.

mean_loss = tf.reduce_mean(sequence_loss)
train_op = tf.train.AdamOptimizer(learning_rate=0.1).minimize(mean_loss)

### Training and print results

In [ ]:

sess = tf.Session()
sess.run(tf.global_variables_initializer())

for i in range(500):
    # result에 batch 별로 들어가있기 때문에, 굉장히 많이 연산될것.
    _, l, results = sess.run(
        [train_op, mean_loss, outputs], feed_dict={X: dataX, Y: dataY})
    for j, result in enumerate(results):
        index = np.argmax(result, axis=1)
        print(i, j, ''.join([char_set[t] for t in index]), l)


### Training and print results
모든 학습이 끝난 후, 결과 출력하기

In [33]:
# Let's print the last char of each result to check it works
results = sess.run(outputs, feed_dict={X: dataX})
for j, result in enumerate(results):
    index = np.argmax(result, axis=1)
    if j is 0:  # print all for the first result to make a sentence
        print(''.join([char_set[t] for t in index]), end='')
    else:
        print(char_set[index[-1]], end='')


g you want to build a ship, don't drum up people together to collect wood and don't assign them tasks and work, but rather teach them to long for the endless immensity of the sea.

### Char-RNN
우리가 배운 것들을 잘 이용하면, shakepear를 가르킬수도 있다..<br>
잘 학습시켜서 shakespear가 쓴 것처럼 쓰게 할 수도 있다.<br>
게다가, 학습데이터가 풍부하다면 우리가 RNN 을 통해 사람이 쓴것처럼 주석도 쓰고, 코드도 작성하게 할 수도 있는 auto coding 도 할 수 있을 것...!

In [34]:
# Let's print the last char of each result to check it works
results = sess.run(outputs, feed_dict={X: dataX})
for j, result in enumerate(results):
    index = np.argmax(result, axis=1)
    if j is 0:  # print all for the first result to make a sentence
        print(''.join([char_set[t] for t in index]), end='')
    else:
        print(char_set[index[-1]], end='')


g you want to build a ship, don't drum up people together to collect wood and don't assign them tasks and work, but rather teach them to long for the endless immensity of the sea.